# Ski Resort Price Analysis

This notebook explores ski resort pricing data along with weather information to identify patterns, trends, and correlations. We'll work through loading, cleaning, and analyzing CSV data to extract meaningful insights about how factors like weather and location impact ski resort prices.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Set visualization styles
plt.style.use("ggplot")
sns.set_palette("viridis")
sns.set_context("notebook")

## 1. Working with CSV Data

### 1.1 Reading and Exploring CSV Data

First, we'll load our datasets and perform some initial exploration to understand what we're working with.

In [ ]:
# Load CSV files
df_prices = pd.read_csv("data/01_ski-prices/prices.csv")
df_weather = pd.read_csv("data/01_ski-prices/weather.csv")

print("Ski Resort Prices - First 5 rows:")
df_prices.head()

In [ ]:
# Get basic information about the price dataset
print("Price dataset info:")
df_prices.info()

In [ ]:
# Statistical summary of the price data
df_prices.describe()

In [ ]:
# Check for missing values in price data
print("Missing values in price data:")
df_prices.isna().sum()

In [ ]:
# Examine the weather data
print("Weather data - First 5 rows:")
df_weather.head()

In [ ]:
# Get basic information about the weather dataset
print("Weather dataset info:")
df_weather.info()

In [ ]:
# Check for missing values in weather data
print("Missing values in weather data:")
df_weather.isna().sum()

### 1.2 Data Distribution Analysis

Let's examine the distribution of our key variables before proceeding with the analysis.

In [ ]:
# Distribution of temperature and precipitation
fig, ax = plt.subplots(3, 1, figsize=(15, 15))

sns.histplot(df_prices["price"], kde=True)
ax[0].set_title("Distribution of Ski Resort Prices")
ax[0].set_xlabel("Price (€)")

sns.histplot(df_weather["temperature"], kde=True, ax=ax[0])
ax[1].set_title("Temperature Distribution")
ax[1].set_xlabel("Temperature (°C)")

sns.histplot(df_weather["precipitation"], kde=True, ax=ax[1])
ax[2].set_title("Precipitation Distribution")
ax[2].set_xlabel("Precipitation (mm)")

plt.tight_layout()
plt.show()

### 1.3 Handling Missing Values

Now we'll handle missing values in our datasets. For some analyses we'll drop them, while for others (like correlation) we'll handle them more carefully.

In [ ]:
# Create cleaned copies for general analysis (dropping missing values)
df_prices_cleaned = df_prices.dropna()
df_weather_cleaned = df_weather.dropna()

# Create copies for correlation analysis (keeping the original structure)
df_prices_for_correlation = df_prices.copy()
df_weather_for_correlation = df_weather.copy()

# Count records before and after cleaning
print(
    f"Price data: {len(df_prices)} rows before cleaning, {len(df_prices_cleaned)} after cleaning"
)
print(
    f"Weather data: {len(df_weather)} rows before cleaning, {len(df_weather_cleaned)} after cleaning"
)

### 1.4 Merging Datasets

Let's combine our price and weather data to analyze how weather conditions might affect pricing.

In [ ]:
# Merge on date and region
df_merged = df_prices_cleaned.merge(
    df_weather_cleaned, on=["date", "region"], how="inner"
)

print(f"Merged dataset has {len(df_merged)} rows")
df_merged.head()

### 1.5 Filtering and Aggregation

Let's examine specific regions and calculate aggregated statistics.

In [ ]:
# Convert date column to datetime format
df_merged["date"] = pd.to_datetime(df_merged["date"])

# List unique regions
print("Available regions:")
df_merged["region"].unique()

In [ ]:
# Filter for a specific resort
kitzbuehl_df = df_merged[df_merged["region"] == "Kitzbuehl"]
print(f"Kitzbuehl dataset has {len(kitzbuehl_df)} rows")
kitzbuehl_df.head()

In [ ]:
# Calculate monthly average price per region
df_monthly = (
    df_merged.groupby([df_merged["date"].dt.to_period("M"), "region"])["price"]
    .mean()
    .reset_index()
)

# Convert period to string for better display
df_monthly["date"] = df_monthly["date"].astype(str)

print("Monthly average prices by region:")
df_monthly.head(10)

In [ ]:
# Calculate price statistics by region
region_stats = (
    df_merged.groupby("region")["price"]
    .agg(["mean", "median", "min", "max", "std"])
    .round(2)
)
print("Price statistics by region:")
region_stats

### 1.6 Visualizing Trends

Let's visualize price trends over time for different regions.

In [ ]:
# Calculate rolling average to smooth trends
df_merged["rolling_avg"] = df_merged.groupby("region")["price"].transform(
    lambda x: x.rolling(7, min_periods=1).mean()
)

# Plot the smoothed trends
plt.figure(figsize=(12, 6))
for region in df_merged["region"].unique():
    subset = df_merged[df_merged["region"] == region]
    plt.plot(subset["date"], subset["rolling_avg"], label=f"{region} (7-day avg)")

plt.xlabel("Date")
plt.ylabel("Rolling Avg Price (€)")
plt.title("Smoothed Ski Resort Price Trends (7-day Rolling Average)")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### 1.7 Correlation Analysis

Let's examine the relationships between price and weather variables.

In [ ]:
# Merge datasets on date and region to analyze correlations
df_correlation = pd.merge(
    df_weather_for_correlation,
    df_prices_for_correlation,
    on=["date", "region"],
    how="inner",  # Keep only rows that match in both datasets
)

# Remove rows with missing values for clean correlation analysis
df_correlation = df_correlation.dropna()

# Add a day of week column to analyze weekday patterns
df_correlation["day_of_week"] = pd.to_datetime(df_correlation["date"]).dt.dayofweek

# Create a 2x2 grid of subplots (one row for each relationship, one column for each region)
fig, axes = plt.subplots(4, 2, figsize=(15, 20))
fig.suptitle("Weather Effects on Ski Prices by Region", fontsize=20, y=1.01)

regions = df_correlation["region"].unique()

# Temperature vs Price - one plot per region
for i, region in enumerate(regions):
    region_data = df_correlation[df_correlation["region"] == region]

    # Temperature plot (top row)
    sns.scatterplot(
        data=region_data,
        x="temperature",
        y="price",
        alpha=0.7,
        color=f"C{i}",
        ax=axes[i, 0],
    )
    axes[i, 0].set_title(f"{region}")
    axes[i, 0].set_xlabel("Temperature (°C)")
    axes[i, 0].set_ylabel("Price (€)")

    # Add regression line
    sns.regplot(
        x="temperature",
        y="price",
        data=region_data,
        scatter=False,
        ax=axes[i, 0],
        color=f"C{i}",
        line_kws={"linestyle": "--"},
    )

    # Precipitation plot (bottom row)
    sns.scatterplot(
        data=region_data,
        x="precipitation",
        y="price",
        alpha=0.7,
        color=f"C{i}",
        ax=axes[i, 1],
    )
    axes[i, 1].set_title(f"{region}")
    axes[i, 1].set_xlabel("Precipitation (mm)")
    axes[i, 1].set_ylabel("Price (€)")

    # Add regression line
    sns.regplot(
        x="precipitation",
        y="price",
        data=region_data,
        scatter=False,
        ax=axes[i, 1],
        color=f"C{i}",
        line_kws={"linestyle": "--"},
    )

plt.tight_layout()
plt.show()

In [ ]:
# Plots showing weekday effect for all regions
fig, axes = plt.subplots(1, 4, figsize=(15, 5))
fig.suptitle("Weekday Effect on Ski Prices by Region", fontsize=16)

for i, region in enumerate(regions):
    sns.boxplot(
        data=df_correlation[df_correlation["region"] == region],
        x="day_of_week",
        y="price",
        ax=axes[i],
        color=f"C{i}",
    )
    axes[i].set_xlabel("Day of Week")
    axes[i].set_ylabel("Price (€)")
    axes[i].set_title(f"{region}")
    axes[i].set_xticks(range(7))
    axes[i].set_xticklabels(["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"])

plt.tight_layout()
plt.show()

## 1.8 Advanced Data Manipulation

Let's apply more advanced data transformations to extract deeper insights from our ski resort data.


In [ ]:
# Convert temperature to Fahrenheit for international comparison
df_merged["temperature_f"] = (
    df_merged["temperature"].apply(lambda x: (x * 9 / 5) + 32).round(1)
)


# Create a temperature category column
def temp_category(temp):
    if temp < -10:
        return "Extremely Cold"
    elif temp < -5:
        return "Very Cold"
    elif temp < 0:
        return "Cold"
    elif temp < 5:
        return "Cool"
    else:
        return "Mild"


df_merged["temp_category"] = df_merged["temperature"].apply(temp_category)

# Count occurrences of each temperature category
print("\nTemperature category distribution:")
print(df_merged.groupby("temp_category").size().sort_values(ascending=False))

# Create a "weekend" flag
df_merged["is_weekend"] = pd.to_datetime(df_merged["date"]).dt.dayofweek >= 5
print("\nWeekend vs. Weekday price comparison:")
print(df_merged.groupby("is_weekend")["price"].agg(["mean", "median", "count"]))

# Calculate price difference from regional average
region_avg = df_merged.groupby("region")["price"].transform("mean")
df_merged["price_vs_region_avg"] = df_merged["price"] - region_avg
print("\nSample of prices compared to regional average:")
print(df_merged[["region", "date", "price", "price_vs_region_avg"]].sample(5))

# Add a column for price per recent snowfall ratio (value metric)
# Avoid division by zero with np.where
df_merged["price_per_snow"] = np.where(
    df_merged["precipitation"] > 0,
    df_merged["price"] / df_merged["precipitation"],
    np.nan,
)

print("\nPrice per snow unit (value metric):")
print(df_merged.groupby("region")["price_per_snow"].mean().sort_values())


## 1.9 Interactive Visualizations with Plotly


In [ ]:
# Create an interactive scatter plot of temperature vs. price
fig = px.scatter(
    df_merged,
    x="temperature",
    y="price",
    color="region",
    size="precipitation",
    hover_data=["date", "temperature_f", "temp_category"],
    labels={"temperature": "Temperature (°C)", "price": "Price (€)"},
)

plt.tight_layout()
fig.update_layout(
    width=1000,
    title="Ski Resort Prices vs. Temperature",
    title_x=0.5,
)
fig.show()


In [ ]:
# Create an interactive time series with dropdown for regions
fig = px.line(
    df_merged,
    x="date",
    y="price",
    color="region",
    line_group="region",
    labels={"date": "Date", "price": "Price (€)"},
)


plt.tight_layout()
fig.update_layout(
    width=1000,
    title="Price Trends Over Time by Region",
    title_x=0.5,
)
fig.show()


In [ ]:
# Create a more complex visualization with multiple data views
fig = make_subplots(
    rows=2,
    cols=2,
    subplot_titles=(
        "Temperature vs Price",
        "Precipitation vs Price",
        "Weekend Price Premium",
        "Price by Temperature Category",
    ),
    specs=[
        [{"type": "scatter"}, {"type": "scatter"}],
        [{"type": "bar"}, {"type": "box"}],
    ],
)

# Temperature vs Price scatter
for region in df_merged["region"].unique():
    region_data = df_merged[df_merged["region"] == region]
    fig.add_trace(
        go.Scatter(
            x=region_data["temperature"],
            y=region_data["price"],
            mode="markers",
            name=region,
            legendgroup=region,
            marker=dict(size=8),
        ),
        row=1,
        col=1,
    )

# Precipitation vs Price scatter
for region in df_merged["region"].unique():
    region_data = df_merged[df_merged["region"] == region]
    fig.add_trace(
        go.Scatter(
            x=region_data["precipitation"],
            y=region_data["price"],
            mode="markers",
            name=region,
            legendgroup=region,
            marker=dict(size=8),
            showlegend=False,
        ),
        row=1,
        col=2,
    )

# Weekend premium bar chart
weekend_premium = (
    df_merged.groupby(["region", "is_weekend"])["price"].mean().reset_index()
)
weekend_premium = weekend_premium.pivot(
    index="region", columns="is_weekend", values="price"
)
weekend_premium["premium"] = weekend_premium[True] - weekend_premium[False]

fig.add_trace(
    go.Bar(
        x=weekend_premium.index, y=weekend_premium["premium"], name="Weekend Premium"
    ),
    row=2,
    col=1,
)

# Box plot by temperature category
for i, region in enumerate(df_merged["region"].unique()):
    region_data = df_merged[df_merged["region"] == region]
    fig.add_trace(
        go.Box(
            x=region_data["temp_category"],
            y=region_data["price"],
            name=region,
            legendgroup=region,
            showlegend=False,
        ),
        row=2,
        col=2,
    )

plt.tight_layout()
fig.update_layout(
    height=800,
    width=1000,
    title_text="Multi-faceted Analysis of Ski Resort Prices",
    title_x=0.5,  # Centers the title
)
fig.show()


## 1.10 Complex Aggregations and Analysis

In [ ]:
# Group by multiple dimensions for deeper insights
complex_agg = (
    df_merged.groupby(["region", "temp_category", "is_weekend"])
    .agg(
        {"price": ["count", "mean", "median", "std"], "precipitation": ["mean", "sum"]}
    )
    .round(2)
)

print("Multi-dimensional aggregation:")
print(complex_agg)

# Find the top 5 most expensive days for each region
top_prices = df_merged.sort_values("price", ascending=False).groupby("region").head(5)
print("\nTop 5 most expensive days by region:")
print(top_prices[["region", "date", "price", "temperature", "precipitation"]])

# Calculate price dynamics (day-to-day changes)
df_merged = df_merged.sort_values(["region", "date"])
df_merged["price_prev_day"] = df_merged.groupby("region")["price"].shift(1)
df_merged["price_change"] = df_merged["price"] - df_merged["price_prev_day"]

print("\nLargest price increases:")
price_increases = df_merged.sort_values("price_change", ascending=False).head(10)
print(price_increases[["region", "date", "price", "price_prev_day", "price_change"]])

# Visual comparison of price distributions
fig, ax = plt.subplots(1, 2, figsize=(15, 6))

# Violin plot showing distribution by region
sns.violinplot(x="region", y="price", data=df_merged, ax=ax[0])
ax[0].set_title("Price Distribution by Region")
ax[0].set_ylabel("Price (€)")
ax[0].set_xlabel("Region")

# Stacked histogram of price by temperature category
temp_order = ["Extremely Cold", "Very Cold", "Cold", "Cool", "Mild"]
sns.histplot(
    data=df_merged,
    x="price",
    hue="temp_category",
    multiple="stack",
    hue_order=temp_order,
    ax=ax[1],
)
ax[1].set_title("Price Distribution by Temperature Category")
ax[1].set_xlabel("Price (€)")

plt.tight_layout()
plt.show()

## 1.11 Saving Processed Data

Finally, let's save our processed dataset for future use.

In [ ]:
merged_df.to_csv("tmp/01_ski-prices/out.csv")